<a href="https://colab.research.google.com/github/nilfredb/SIC25-DataGeeks/blob/main/Ecolens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit pyngrok pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.1 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()  # selecciona Unemployment_rate_dataset.csv

Saving Unemployment_rate_dataset.csv to Unemployment_rate_dataset.csv


In [3]:
%%writefile econlens_app_streamlit.py
# econlens_app_streamlit.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import io
import os

st.set_page_config(page_title="EconLens — Análisis Económico", page_icon="📊", layout="wide")

st.title("📊 EconLens — Plataforma Interactiva de Análisis Económico")
st.markdown("Esta aplicación muestra cómo el **crecimiento económico (PIB per cápita)** no siempre se traduce en **reducción del desempleo**. Datos: Banco Mundial (ejemplo).")

@st.cache_data
def load_data():
    """Carga los datasets: local para desempleo y GitHub para PIB (si está disponible)."""
    try:
        # 1) Archivo local: Unemployment_rate_dataset.csv (debes subirlo en Colab)
        local_unemp = "Unemployment_rate_dataset.csv"
        if not os.path.exists(local_unemp):
            st.error(f"❌ No se encontró '{local_unemp}' en el directorio actual. Súbelo (Files -> Upload) o monta Drive.")
            return None

        unemp = pd.read_csv(local_unemp)

        # 2) Dataset grande (PIB per cápita) desde GitHub (URL de ejemplo)
        github_url = "https://raw.githubusercontent.com/nilfredb/SIC25-DataGeeks/65b43ad1719ec4cb996c9d20f407d904a185671c/pib_per_capita_countries_dataset.csv"
        try:
            response = urllib.request.urlopen(github_url, timeout=15)
            gdp_data = response.read().decode("utf-8")
            gdp_countries = pd.read_csv(io.StringIO(gdp_data))
        except Exception as e:
            st.warning(f"⚠️ No se pudo descargar el CSV de PIB desde GitHub: {e}\nSi tienes tu propio CSV, súbelo y cambia la variable github_url.")
            return None

        # Transformaciones
        year_cols = [col for col in unemp.columns if col.isdigit()]
        if not year_cols:
            st.error("❌ No se encontraron columnas de años en el archivo de desempleo. Revisa el formato del CSV.")
            return None

        unemp_long = unemp.melt(
            id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
            value_vars=year_cols,
            var_name='year',
            value_name='unemployment_rate'
        ).dropna(subset=['unemployment_rate'])
        unemp_long['year'] = unemp_long['year'].astype(int)
        unemp_long = unemp_long.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'})

        gdp_countries = gdp_countries[['country_code', 'country_name', 'year', 'gdp_per_capita']].copy()
        gdp_countries['year'] = pd.to_numeric(gdp_countries['year'], errors='coerce').astype('Int64')
        gdp_countries['gdp_per_capita'] = pd.to_numeric(gdp_countries['gdp_per_capita'], errors='coerce')
        gdp_countries = gdp_countries.dropna(subset=['year']).copy()
        gdp_countries['year'] = gdp_countries['year'].astype(int)

        merged = pd.merge(unemp_long, gdp_countries[['country_code', 'year', 'gdp_per_capita']],
                          on=['country_code', 'year'], how='inner')
        return merged

    except Exception as e:
        st.error(f"Error al cargar datos: {e}")
        return None

data = load_data()
if data is None or data.empty:
    st.stop()

# SIDEBAR (Filtros)
st.sidebar.header("🎛️ Controles de Filtrado")
all_countries = sorted(data['country_name'].unique())
selected_countries = st.sidebar.multiselect("Selecciona uno o varios países", options=all_countries, default=['Dominican Republic', 'United States', 'Japan', 'South Africa', 'Spain'])
year_min = int(data['year'].min())
year_max = int(data['year'].max())
year_range = st.sidebar.slider("Rango de años", min_value=year_min, max_value=year_max, value=(1991, 2023))

# Filtrado
filtered = data[(data['country_name'].isin(selected_countries)) & (data['year'] >= year_range[0]) & (data['year'] <= year_range[1])]
if filtered.empty:
    st.warning("⚠️ No hay datos para los filtros seleccionados.")
    st.stop()

# Estadísticas
corr = filtered['gdp_per_capita'].corr(filtered['unemployment_rate'])
st.sidebar.markdown(f"**📈 Correlación PIB–Desempleo:** `{corr:.3f}`")
if corr < -0.3:
    conclusion = "📉 Relación inversa fuerte: cuando el PIB sube, el desempleo tiende a bajar."
elif corr < 0:
    conclusion = "📊 Relación inversa débil: ligera tendencia a menos desempleo con mayor PIB."
elif corr > 0.3:
    conclusion = "📈 Relación directa fuerte: el desempleo aumenta con el PIB (paradoja económica)."
elif corr > 0:
    conclusion = "📊 Relación directa débil: ligera tendencia a mayor desempleo con más PIB."
else:
    conclusion = "➖ Sin correlación clara entre PIB y desempleo."
st.info(conclusion)

# Visualizaciones
tab1, tab2, tab3 = st.tabs(["📈 Evolución del Desempleo", "🔵 PIB vs Desempleo", "📊 Análisis Dual (País)"])
with tab1:
    st.subheader("Evolución del Desempleo por País")
    fig, ax = plt.subplots(figsize=(10,5))
    for c in selected_countries:
        country_data = filtered[filtered['country_name'] == c].sort_values('year')
        ax.plot(country_data['year'], country_data['unemployment_rate'], label=c)
    ax.set_xlabel("Año"); ax.set_ylabel("Tasa de Desempleo (%)"); ax.set_title("Evolución del Desempleo (1991–2023)")
    ax.legend(); ax.grid(True)
    st.pyplot(fig)
with tab2:
    st.subheader("Relación entre PIB per cápita y Desempleo")
    fig, ax = plt.subplots(figsize=(8,5))
    for c in selected_countries:
        country_data = filtered[filtered['country_name'] == c]
        ax.scatter(country_data['gdp_per_capita'], country_data['unemployment_rate'], label=c, alpha=0.7)
    ax.set_xscale('log'); ax.set_xlabel("PIB per cápita (US$, escala log)"); ax.set_ylabel("Desempleo (%)"); ax.set_title("Relación PIB vs Desempleo")
    ax.legend(); ax.grid(True)
    st.pyplot(fig)
with tab3:
    if len(selected_countries) > 0:
        country = selected_countries[0]
        st.subheader(f"Evolución PIB y Desempleo — {country}")
        country_data = filtered[filtered['country_name'] == country].sort_values('year')
        fig, ax1 = plt.subplots(figsize=(10,5))
        ax1.set_xlabel("Año"); ax1.set_ylabel("Desempleo (%)", color='tab:red')
        ax1.plot(country_data['year'], country_data['unemployment_rate'], color='tab:red', marker='o', label='Desempleo'); ax1.tick_params(axis='y', labelcolor='tab:red')
        ax2 = ax1.twinx(); ax2.set_ylabel("PIB per cápita (US$)", color='tab:blue')
        ax2.plot(country_data['year'], country_data['gdp_per_capita'], color='tab:blue', marker='s', label='PIB per cápita'); ax2.tick_params(axis='y', labelcolor='tab:blue')
        fig.tight_layout(); st.pyplot(fig)
    else:
        st.info("Selecciona al menos un país para ver el análisis dual.")

st.download_button(label="💾 Descargar datos filtrados (CSV)", data=filtered.to_csv(index=False).encode('utf-8'), file_name="econlens_filtered_data.csv", mime="text/csv")


Writing econlens_app_streamlit.py


In [6]:
from pyngrok import ngrok
ngrok.set_auth_token("33n0QLQfMgpbF3OVghxpBgO8b7I_EiJUcQ4ugXrna51ivusv")

In [7]:
# 1) iniciar la app en background (colab)
get_ipython().system_raw('nohup streamlit run econlens_app_streamlit.py --server.port 8501 --server.headless true >/content/streamlit.log 2>&1 &')

# 2) abrir túnel con pyngrok
from pyngrok import ngrok
# Si tienes token (recomendado): ngrok.set_auth_token("TU_NGROK_AUTHTOKEN")
public_url = ngrok.connect(8501)
print("Esta es nuestra interfaz:", public_url)


Abre esta URL en tu navegador: NgrokTunnel: "https://anh-untinseled-gina.ngrok-free.dev" -> "http://localhost:8501"
